In [1]:
import urllib
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import openapi

In [2]:
import requests

# ✅ API 키 및 URL 설정
s_key = 'l53cN3mbhsudCVoI7zttwvpHkgCTM8on0LpEaT2FZT1/6+weHhppLkhYpjiYtdPL3/h7rqYRcFNBJQCklXUZUQ=='
url = 'http://apis.data.go.kr/B551172/Gastric03/gsJobByType'

# ✅ 기본 파라미터 설정 (numOfRows를 정수로 변경)
params = {
    'serviceKey': s_key,
    'type': 'json',
    'numOfRows': 100,  # 🔹 문자열이 아닌 정수로 변경
    'centerNm': '국립암센터',
    'fromYear': '2010',
    'toYear': '2019',
    'pageNo': 1
}

all_data = []

# 1️⃣ 먼저 첫 번째 요청을 보내서 totalCount 확인
response = requests.get(url, params=params)

if response.status_code == 200:
    resp_dict = response.json()

    # 🔹 totalCount가 없을 경우 기본값 0으로 설정 (오류 방지)
    total_count = int(resp_dict.get('totalCount', 0) or 0)  
    items_per_page = params['numOfRows']  
    total_pages = (total_count // items_per_page) + (1 if total_count % items_per_page else 0)

    print(f"📌 총 데이터 개수: {total_count}, 가져올 페이지 수: {total_pages}")

    # ✅ 데이터가 없을 경우 메시지 출력 후 종료
    if total_count == 0:
        print("⚠️ 가져올 데이터가 없습니다.")
    else:
        # 2️⃣ total_pages 만큼 반복하여 모든 데이터를 가져오기
        for i in range(1, total_pages + 1):
            page_params = params.copy()  # 🔹 원본 params 보호
            page_params['pageNo'] = i  

            response = requests.get(url, params=page_params)

            if response.status_code == 200:
                resp_dict = response.json()
                items = resp_dict.get('items', [])  # 🔹 데이터가 없을 경우 빈 리스트 반환
                all_data.extend(items)
                print(f"📄 페이지 {i}/{total_pages} 처리 완료! (누적 {len(all_data)}개)")
            else:
                print(f"❌ 데이터 요청 실패 (페이지 {i})")

        print(f"✅ 총 {len(all_data)}개의 데이터 수집 완료!")

else:
    print(f"❌ API 요청 실패 - HTTP 상태 코드: {response.status_code}")


df_job = pd.DataFrame(all_data)


📌 총 데이터 개수: 1580, 가져올 페이지 수: 16
📄 페이지 1/16 처리 완료! (누적 100개)
📄 페이지 2/16 처리 완료! (누적 200개)
📄 페이지 3/16 처리 완료! (누적 300개)
📄 페이지 4/16 처리 완료! (누적 400개)
📄 페이지 5/16 처리 완료! (누적 500개)
📄 페이지 6/16 처리 완료! (누적 600개)
📄 페이지 7/16 처리 완료! (누적 700개)
📄 페이지 8/16 처리 완료! (누적 800개)
📄 페이지 9/16 처리 완료! (누적 900개)
📄 페이지 10/16 처리 완료! (누적 1000개)
📄 페이지 11/16 처리 완료! (누적 1100개)
📄 페이지 12/16 처리 완료! (누적 1200개)
📄 페이지 13/16 처리 완료! (누적 1300개)
📄 페이지 14/16 처리 완료! (누적 1400개)
📄 페이지 15/16 처리 완료! (누적 1500개)
📄 페이지 16/16 처리 완료! (누적 1580개)
✅ 총 1580개의 데이터 수집 완료!


In [3]:
df_job = df_job.iloc[:, [1,2,3,4,5,-2]]
df_job

centerNm critYr ptAge ptSexCd statsTrgtNm ptCntNmvl
0       국립암센터   2010    51       M          기타         3
1       국립암센터   2016    74       F          주부         5
2       국립암센터   2017    70       M         전문직         1
3       국립암센터   2013    48       F         자유업         1
4       국립암센터   2016    63       M          기타         2
...       ...    ...   ...     ...         ...       ...
1575    국립암센터   2010    64       M          기타         1
1576    국립암센터   2016    43       M          무직         1
1577    국립암센터   2011    65       F          주부         1
1578    국립암센터   2012    81       M         자유업         1
1579    국립암센터   2014    48       F          교사         1

[1580 rows x 6 columns]

In [4]:
df_job.rename(columns={'centerNm':'센터 이름', 'ptAge':'나이', 'ptSexCd':'성별', 'statsTrgtNm':'직업', 'ptCntNmvl':'환자수'}, inplace=True)
df_job

센터 이름 critYr  나이 성별   직업 환자수
0     국립암센터   2010  51  M   기타   3
1     국립암센터   2016  74  F   주부   5
2     국립암센터   2017  70  M  전문직   1
3     국립암센터   2013  48  F  자유업   1
4     국립암센터   2016  63  M   기타   2
...     ...    ...  .. ..  ...  ..
1575  국립암센터   2010  64  M   기타   1
1576  국립암센터   2016  43  M   무직   1
1577  국립암센터   2011  65  F   주부   1
1578  국립암센터   2012  81  M  자유업   1
1579  국립암센터   2014  48  F   교사   1

[1580 rows x 6 columns]

In [5]:
df_job['환자수'] = pd.to_numeric(df_job['환자수'], errors='coerce')

df_job['환자수'] = df_job['환자수'].fillna(0).astype(int)

job_count = df_job.groupby(df_job['직업'].fillna('무응답'))['환자수'].sum()

print(job_count)

직업
교사      24
군인       4
기타     519
무응답     29
무직     853
자유업    322
전문직     60
주부     384
학생       6
회사원    283
Name: 환자수, dtype: int64


In [6]:
import matplotlib.font_manager as fm

# 한글 폰트 설정 (예: 맑은 고딕)
plt.rcParams['font.family'] = 'Malgun Gothic'

# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 파이차트 그리기
plt.figure(figsize=(6, 6))
plt.pie(job_count.values, labels=job_count.index, autopct='%1.1f%%', startangle=140)
plt.title('직업 분포')
plt.axis('equal')
plt.show()